In [ ]:

package = ['boto3','es_lemmatizer','pyodbc','prince']
f= open("paquetes.txt","w+")

def install_txt(package):
  f= open("paquetes.txt","w+")
  for i in package:
    print(i)
    

    try:
        __import__(i)
        print('ya instalado {}'.format(i))
    except ImportError:
        # main(['install', package]) 
        f.write("{} \r\n".format(i))
        print('No instalado {}'.format(i))


install_txt(package)  
  

boto3
ya instalado boto3
es_lemmatizer
ya instalado es_lemmatizer
pyodbc
ya instalado pyodbc
prince
ya instalado prince


In [ ]:
!pip install -r paquetes.txt

#**Librerias**

In [ ]:
import re
import numpy as np
import pandas as pd
from pathlib import Path
from PIL import Image
from wordcloud import WordCloud, STOPWORDS, ImageColorGenerator
import nltk
nltk.download('stopwords')
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
import unicodedata
from es_lemmatizer import lemmatize
import spacy
import re
import pyodbc
#import stanza


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


#**Funciones**

In [ ]:
#### Rutas a carpetas
ecopetrol  = Path('Base_Campaña_Personal.csv')
ecopetrol_2  = Path('Base_Campaña_OperacionesFinacieras.csv')
ecopetrol_3  = Path('Base_general_extraida_P.csv')

#palabras que se pueden ignorar durante el analisis
ADDITIONAL_STOPWORDS = ['buenas', 'tardes', 'buenos', 'dia', 'necesito'
                        ,'nueva', 'eps', 'saber', 'necesito', 'quiero'
                        ,'dia','dias', 'buena', 'tarde', 'favor'
                        , 'solicitar', 'pedir', 'sacar', 'si','una','unas','puedo','debo', 'hacer'
                        ,'hola', 'dice', 'q', 'deseo', '19', 'hace', 'gracias'
                        , 'pueden', 'autorizacion'
                        ,'ok','senores','martha']#,'autorizacion']


grupo_1 = 'observaciones,observacion,senor,senora,ano,2020,maria,quiere,ecopetrol,ecp,luis,2021,febrero,jimmy,sebastian,persn,gordillo,sebastian,gamboa,requiere,siguientes,saludo,cordial,usuario,maria,atento,quedo,ecp'
grupo_2 = 'legalizar,realizo,jose,sr,ustedes,agradezco,realizar,desea,solicitando,cedula,cc,ciudadania,juan,siguiente,siguientes,beneficiarioa'
grupo_3 = 'preguntando,mes,siempre,acostumada,respuesta,documentacion,fin,dirijo,amable,solicitamos,colaboracion,respuesta,sra,sr,septieme,debe,permitimos,informarle,debe,mb,informa,andrea,sabogal,c,andrea,angelica,sabogal,rodriguez,dar,segun,encargada,com,validar,envio,caso,respuesa,solicita,solicito,comunica,puede,novieme,documento,documentos,buen,encargada,fecha,personal,michelle,yuliana,herrera,parte,ditta,dario,luna,santo,tomas,bucaramanga,camilo,franco,rivera,personal,proceso,area,espero,mas,solo,patricia,acevedo,villamizar,jorge,emilio,espinoza,cantillo,jairo,mpara,oficinavirtualdepersonalecopetrolcomcoasunto,p,presente,medio,irma,lizette,diaz,figueroa,m,oficinavirtualecopetrolcomco,asunto,20203,20202,4,gutierrez,salome' 
#Diccionario para reemplazar palabras
dic = {'beneficiaria':'beneficiario', 'autorizaciones' : 'autorizacion', 'citas' :'cita','incapacidades':'incapacidad' 
       , 'radicacion':'radicar', 'transcripcion':'transcribir', 'medicamentos':'medicamento'
       ,'solicito':'solicitud', 'autorizacion':'autorizar', 'comunicarme':'hablar'
       , 'examenes' : 'examen', 'acesor':'asesor', 'consulta':'consultar', 'agente' : 'asesor', 'nesecito':'necesito'}

#stopwords
stop_words = nltk.corpus.stopwords.words('spanish') + ADDITIONAL_STOPWORDS + grupo_1.split(',')+ grupo_2.split(',') + grupo_3.split(',')



#convierte en un solo texto la columna de motivos
def text (column, dic):
    column = column.astype(str)
    texto_base = " ".join(motivo for motivo in column)
    text = (unicodedata.normalize('NFKD', texto_base).encode('ascii', 'ignore').decode('utf-8', 'ignore').lower())
    text = re.sub(r"\bmedicas\b","medica",text)
    for i, j in dic.items():
        text = text.replace(i, j)
        
    return text
    
def listas (column, dic):
    Lista = []
    column = columna.astype(str)
    for motivo in column:
        inicial_answer = str(motivo)
        for i, j in dic.items():
            ini_answer = inicial_answer.replace(i, j)
        answer = (unicodedata.normalize('NFKD', ini_answer).encode('ascii', 'ignore').decode('utf-8', 'ignore').lower())
        answer = re.sub(r"\bmedicas\b","medica",answer)
        answer = re.sub(r"\bmedicamentos\b","medicamento",answer)
        answer = re.sub(r'[^\w\s]','', answer).split()
        words = [word for word in answer if word not in stop_words]
        Lista.append(words)
    return Lista
    
#crea nube de palabras
def wordcloud(text, nombre, stopWords):
    wordcloud = WordCloud(stopwords=stop_words+stopWords, background_color="white", width=1600, height=800,min_word_length =3 ).generate(text)
    plt.figure( figsize=(15,10) )
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.show()
    plt.savefig("word_cloud_"+str(nombre)+".png") #dpi = 300)    

#crea n-gramas   
def ngrams(text, n, top,stopWords):
    #wnl = nltk.stem.WordNetLemmatizer()
    texto = re.sub(r'[^\w\s]','', text).split()
    stopwords = stop_words + stopWords
    words = [word for word in texto if word not in stopwords]
    
    n_grams_series = ((pd.Series(nltk.ngrams(words,n)).value_counts())[:top])
    plot = n_grams_series.sort_values().plot.barh(color ="tomato", width = 0.6, figsize = (20,18), edgecolor='grey')   #color=(0.2, 0.4, 0.6, 0.6)
    plot.set_xlabel('Frecuencia',  fontname="Calibri", fontsize=35)
    plot.set_title('TOP ' + str(top) +' de '+str(n)+'-GRAMAS QUE APARECEN CON MAYOR FRECUENCIA', fontname="Calibri", fontsize=40)
    plot.title.set_position([.5, 1.05])
    for tick in  plot.get_xticklabels():
        tick.set_fontname("Calibri")
        tick.set_fontsize(35)
    for tick in  plot.get_yticklabels():
        tick.set_fontname("Calibri")
        tick.set_fontsize(35)
    #plt.savefig("graph.png") #dpi = 300)
    plt.figure( figsize=(15,10) )
    plt.show()
    return n_grams_series
    #if n == 1 :
    #    return n_grams_series.to_frame().reset_index().rename(columns = {'index':'palabras', 0:'Conteo'})
    #elif n == 2:
    #    return n_grams_series.to_frame().reset_index().rename(columns = {'index':'Bigramas', 0:'Conteo'})
    #elif n > 2:
    #    return n_grams_series.to_frame().reset_index().rename(columns = {'index':'gramas', 0:'Conteo'})





# **EXPLORACIÓN DE TEXTO - BENEFICIOS EDUCATIVOS**

##**Extracción**

In [ ]:
beneficios = pd.read_csv(ecopetrol)

beneficios.columns

Index(['fgs_FechaInicio', 'fgs_FechaFin', 'Cumple_Objetivo_RegistradoEscalado',
       'fdp_NombreGrupoSolucion', 'fgs_ObservacionCierre',
       'fgs_CiudadDireccion_01', 'fgs_SiglasDocumentoContacto',
       'fgs_Direccion_01', 'Cumple_Objetivo_SolucionadoCerrado',
       'NoReprocesar', 'fgs_ObservacionCreacion', 'fgs_Telefono_01',
       'fgs_Email_01', 'fmf_Canal', 'fgs_Nivel2', 'GrupoRolSolucion',
       'fdp_NombreUsuarioCreacion', 'fdp_NombreEstado', 'GrupoRolCreacion',
       'fgs_NumeroDocumContacto', 'SistemaPago', 'Cumple_Objetivo_EnNivel2',
       'fdp_NombreUsuarioSolucion', 'fgs_DepartamentoDireccion_01',
       'fdp_NombreUsuarioGestionActual', 'fgs_NombreCompletoContacto',
       'fgs_Tipificación', 'fgs_PaisDireccion_01', 'ude_Regional',
       'fgs_ObservacionSolucion', 'fgs_Nivel3', 'fgs_IdGestion',
       'Cumple_Objetivo_SolicitadoRegistrado'],
      dtype='object')

In [ ]:
beneficios.columns

Index(['fgs_FechaInicio', 'fgs_FechaFin', 'Cumple_Objetivo_RegistradoEscalado',
       'fdp_NombreGrupoSolucion', 'fgs_ObservacionCierre',
       'fgs_CiudadDireccion_01', 'fgs_SiglasDocumentoContacto',
       'fgs_Direccion_01', 'Cumple_Objetivo_SolucionadoCerrado',
       'NoReprocesar', 'fgs_ObservacionCreacion', 'fgs_Telefono_01',
       'fgs_Email_01', 'fmf_Canal', 'fgs_Nivel2', 'GrupoRolSolucion',
       'fdp_NombreUsuarioCreacion', 'fdp_NombreEstado', 'GrupoRolCreacion',
       'fgs_NumeroDocumContacto', 'SistemaPago', 'Cumple_Objetivo_EnNivel2',
       'fdp_NombreUsuarioSolucion', 'fgs_DepartamentoDireccion_01',
       'fdp_NombreUsuarioGestionActual', 'fgs_NombreCompletoContacto',
       'fgs_Tipificación', 'fgs_PaisDireccion_01', 'ude_Regional',
       'fgs_ObservacionSolucion', 'fgs_Nivel3', 'fgs_IdGestion',
       'Cumple_Objetivo_SolicitadoRegistrado'],
      dtype='object')

##**TIPOS DE OBSERVACION**

In [ ]:
print ('TOTAL DE REGISTROS: ', beneficios.shape[0])

TOTAL DE REGISTROS:  13748


In [ ]:
df= pd.read_csv(ecopetrol_3, low_memory =False)

df.columns

Obs_cols = [col for col in df.columns if 'Observacion' in col]
Obs_cols

df_observaciones = df[Obs_cols]

Faltantes = df_observaciones.isnull().sum().to_frame('cantidad_faltantes').reset_index().rename(columns = {'index':'observaciones'})
Faltantes['% no vacio'] = round((df_observaciones.shape[0]-Faltantes['cantidad_faltantes'])*100/ df_observaciones.shape[0],1)
Faltantes['% vacio'] = round((Faltantes['cantidad_faltantes'])*100/ df_observaciones.shape[0],1)

Faltantes

,observaciones,cantidad_faltantes,% no vacio,% vacio
0,fgs_ObservacionCreacion,0,100.0,0.0
1,fgs_ObservacionEscalamientoN2,10752,21.8,78.2
2,fgs_ObservacionEscalamientoN3,13748,0.0,100.0
3,fgs_ObservacionDevolucion,13527,1.6,98.4
4,fgs_ObservacionSolucion,29,99.8,0.2
5,fgs_ObservacionReapertura,13747,0.0,100.0
6,fgs_ObservacionCierre,30,99.8,0.2
7,fgs_ObservacionGestionActual,0,100.0,0.0
8,fgs_ObservacionEscalamientoN4,13748,0.0,100.0


##**Grupos**

In [ ]:
GRUPOS = ['1. Legalización','1. Legalización','2. Solicitud de educación inclusiva'
          ,'3. Anticipo','3. Anticipo','3. Anticipo','3. Anticipo'
          ,'4. Consulta','4. Consulta','4. Consulta','4. Consulta','4. Consulta'
          ,'4. Consulta','4. Consulta','5. Soporte Plataforma'
          ,'6. Reintegro','6. Reintegro','7. Inscripción','7. Inscripción','7. Inscripción']

NIVELES = ['Legal / Reconocimiento Pensionados (Incluye Sustitutos)'
           ,'Legal / Reconocimiento Trabajadores (Incluye Bachiller ECP)'
           ,'Solicitud de educación inclusiva'
           ,'Anticipo Pensionados Educación Inclusiva','Anticipo Trabajadores Educación Inclusiva'
            ,'Anticipo Pensionados (Incluye Sustitutos)','Anticipo Trabajadores (Incluye BXC)'
            ,'Consulta Bachiller Ecopetrol (Incapacidad médica y/o Fuerza Mayor)'
            ,'Consulta de información relacionadas con reembolsos'
            ,'Consulta especialista beneficios atención PQRS'
            ,'Consulta sobre liquidación/pago préstamo para educación'
            ,'Consulta Titulares Beneficio Eductivo - Desvinculados'
            ,'Consultas Titulares Beneficio Educativo (No Incluye Bachiller ECP)'
            ,'Solicitud / Consulta / revisión pagos colegios propiedad de Ecopetrol S.A.'
            ,'Soporte Plataforma para Instituciones Educativas vía TIB'
            ,'Reintegro de dinero Pensionados Educación (Incluye sustitutos y Auto)'
            ,'Reintegro de dinero trabajadores Educación (Incluye Bachiller ECP)'
            ,'Inscripción casos especiales comité de educación convencional activos'
            ,'Inscripción casos especiales comité educación convencional pensionados'
            ,'Inscripción solicitud becas Comité de Educación Convencional Activos']

dic_grupos = dict(zip(NIVELES,GRUPOS))

In [ ]:
beneficios['GRUPOS'] = beneficios['fgs_Nivel3']
beneficios['GRUPOS'].replace(dic_grupos, inplace=True)

##**Palabras**

In [ ]:
caracterizacion_palabras = pd.read_excel('Caracterizacion_inicial_palabras.xlsx', sheet_name = 'Beneficios-obsCreacion')

caracterizacion_palabras.columns

Index(['nivel3', 'palabras_clave', 'bigramas', 'trigramas', 'Palabras ',
       'Observaciones ', 'Palabras adicionales'],
      dtype='object')

In [ ]:
caracterizacion_palabras = caracterizacion_palabras.replace(regex= [r"\(", r"\),", r"\[", r"\]", r"\'", r","] , value="")

In [ ]:
caracterizacion_palabras['Palabras '] = caracterizacion_palabras['Palabras '].str.strip()

In [ ]:
caracterizacion_palabras.rename(columns = {'Palabras ':'StopWords'}, inplace = True)

In [ ]:
df_total = beneficios
caracterizacion_palabras['fgs_Nivel3'] = list(df_total['fgs_Nivel3'].unique())

In [ ]:
list(df_total['fgs_Nivel3'].unique())

['Legal / Reconocimiento Pensionados (Incluye Sustitutos)',
 'Legal / Reconocimiento Trabajadores (Incluye Bachiller ECP)',
 'Solicitud de educación inclusiva',
 'Anticipo Pensionados Educación Inclusiva',
 'Anticipo Trabajadores Educación Inclusiva',
 'Anticipo Pensionados (Incluye Sustitutos)',
 'Anticipo Trabajadores (Incluye BXC)',
 'Consulta Bachiller Ecopetrol (Incapacidad médica y/o Fuerza Mayor)',
 'Consulta de información relacionadas con reembolsos',
 'Consulta especialista beneficios atención PQRS',
 'Consulta sobre liquidación/pago préstamo para educación',
 'Consulta Titulares Beneficio Eductivo - Desvinculados',
 'Consultas Titulares Beneficio Educativo (No Incluye Bachiller ECP)',
 'Solicitud / Consulta / revisión pagos colegios propiedad de Ecopetrol S.A.',
 'Soporte Plataforma para Instituciones Educativas vía TIB',
 'Reintegro de dinero Pensionados Educación (Incluye sustitutos y Auto)',
 'Reintegro de dinero trabajadores Educación (Incluye Bachiller ECP)',
 'Inscripc

In [ ]:
caracterizacion_palabras[['fgs_Nivel3', 'StopWords', 'Observaciones ', 'Palabras adicionales']]

,fgs_Nivel3,StopWords,Observaciones,Palabras adicionales
0,Legal / Reconocimiento Pensionados (Incluye Su...,plan educacional solicitud legalizacion recono...,El proceso es Reconocimiento o Legalización en...,Ratificación paz y salvo designado beneficiari...
1,Legal / Reconocimiento Trabajadores (Incluye B...,solicitud plan educacional legalizacion recono...,El proceso es Reconocimiento o Legalización en...,Ratificación paz y salvo designado beneficiari...
2,Solicitud de educación inclusiva,gestion area escalo tramite encargada solicitu...,Toda la información y validaciones acerca del ...,Repetición Año concepto evaluación psicodiagno...
3,Anticipo Pensionados Educación Inclusiva,anticipo inclusiva plan educacional tramite ge...,Proceso de entrega de dinero previo al pago op...,Repetición Año concepto evaluación psicodiagno...
4,Anticipo Trabajadores Educación Inclusiva,plan correspondiente educacion 2020 observacio...,Proceso de entrega de dinero previo al pago op...,Ratificación paz y salvo designado beneficiari...
5,Anticipo Pensionados (Incluye Sustitutos),anticipo solicitud plan educacional informacio...,Proceso de entrega de dinero previo al pago op...,Ratificación paz y salvo designado beneficiari...
6,Anticipo Trabajadores (Incluye BXC),anticipo solicitud plan educacional informacio...,Proceso de entrega de dinero previo al pago op...,Ratificación paz y salvo designado beneficiari...
7,Consulta Bachiller Ecopetrol (Incapacidad médi...,anticipo colombia formato beneficiarias perde...,Consultas varias de beneficiarios de bachiller...,pagare derechos grado pasantias tiempo repor...
8,Consulta de información relacionadas con reemb...,plan educacional pago caso descuento comunica ...,Consultas varias de beneficiarios acerca de lo...,prestamo pse auxilio salario vacaciones an...
9,Consulta especialista beneficios atención PQRS,adjunta colombia atuesta solicitud comunicaci...,Consultas especificas de OPC,adjutar quejas solicitud enviar documentos c...


In [ ]:
palabras = caracterizacion_palabras[caracterizacion_palabras['fgs_Nivel3'] == 'Legal / Reconocimiento Trabajadores (Incluye Bachiller ECP)'].reset_index()

In [ ]:
palabras

,index,nivel3,palabras_clave,bigramas,trigramas,StopWords,Observaciones,Palabras adicionales,fgs_Nivel3
0,1,Legal / Reconocimiento Trabajadores Incluye Ba...,solicitud plan educacional legalizacion recono...,plan educacional oficina virtual solicita info...,legalizacion plan educacional reconocimiento p...,solicitud plan educacional legalizacion recono...,El proceso es Reconocimiento o Legalización en...,Ratificación paz y salvo designado beneficiari...,Legal / Reconocimiento Trabajadores (Incluye B...


In [ ]:
palabras['StopWords'].str.split(' ')

0    [solicitud, plan, educacional, legalizacion, r...
Name: StopWords, dtype: object

## **NUBES DE PALABRAS Y PALABRAS CLAVES POR NIVEL 3 - OBSERVACIONES DE CREACION**

###**NIVEL 3**

In [ ]:
beneficios.fgs_Nivel3.value_counts()

Legal / Reconocimiento Trabajadores (Incluye Bachiller ECP)                   4195
Legal / Reconocimiento Pensionados (Incluye Sustitutos)                       2415
Anticipo Trabajadores (Incluye BXC)                                           1807
Anticipo Pensionados (Incluye Sustitutos)                                     1586
Consultas Titulares Beneficio Educativo (No Incluye Bachiller ECP)            1225
Soporte Plataforma para Instituciones Educativas vía TIB                      1121
Solicitud de educación inclusiva                                               414
Inscripción solicitud becas Comité de Educación Convencional Activos           353
Reintegro de dinero trabajadores Educación (Incluye Bachiller ECP)             303
Inscripción casos especiales comité de educación convencional activos           99
Reintegro de dinero Pensionados Educación (Incluye sustitutos y Auto)           58
Consulta Titulares Beneficio Eductivo - Desvinculados                           52
Soli

In [ ]:
mayores_volumenes = beneficios.fgs_Nivel3.value_counts().to_frame('registros').reset_index().rename(columns = {'index':'nivel3'})
mayores_volumenes = mayores_volumenes[mayores_volumenes['registros']>1000] 

###**Grupos-nivel**

In [ ]:
beneficios.GRUPOS.value_counts()

1. Legalización                        6610
3. Anticipo                            3428
4. Consulta                            1348
5. Soporte Plataforma                  1121
7. Inscripción                          466
2. Solicitud de educación inclusiva     414
6. Reintegro                            361
Name: GRUPOS, dtype: int64

### **NIVELES CON MAYOR VOLUMEN**

In [ ]:
df_total = beneficios
nivel3 = []
palabras_clave = []
bigramas = []
trigramas = []

for i in list(mayores_volumenes['nivel3'].unique()):
    df_texto=  df_total[df_total['fgs_Nivel3'] == i]
    palabras = caracterizacion_palabras[caracterizacion_palabras['fgs_Nivel3'] == i].reset_index()
    df_texto['fgs_ObservacionCreacion'] = df_texto['fgs_ObservacionCreacion'].str.replace('\d+', '')
    texto = text(df_texto['fgs_ObservacionCreacion'], dic)
    texto = texto.replace('\d+', '')#replace('\br','')
    print(i.upper(),' Total Registros: ',df_texto.shape[0])
    StopWords = palabras['StopWords'].str.split(' ')[0]
    wordcloud(texto, 'seis', StopWords)
    nivel3.append(i)
    palabras_clave.append(list(ngrams(texto, 1, 10, StopWords).to_frame().index))
    bigramas.append(list(ngrams(texto, 2, 10, StopWords).to_frame().index))
    trigramas.append(list(ngrams(texto, 3, 10, StopWords ).to_frame().index))
    print('-----------------------------------------------------------------------------------------------')
    

LEGAL / RECONOCIMIENTO TRABAJADORES (INCLUYE BACHILLER ECP)  Total Registros:  4195


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


TypeError: ignored

### **GRUPOS**

In [ ]:
StopWordGroups = str(caracterizacion_palabras['StopWords'].unique())
StopWordGroups = StopWordGroups.replace("[","").replace("'","").replace("\n","").replace("]","").replace(")","")#.replace(r"\)","").replace(r"\[","").replace(r"\]","").replace(r"'\n '","")
SW_grupos = StopWordGroups.split(' ')


In [ ]:
palabras_adicionales = ['pronta','pronto','respuesta']

In [ ]:
for i in list(df_total['GRUPOS'].unique()):
    df_texto=  df_total[df_total['GRUPOS'] == i]
    #palabras = caracterizacion_palabras[caracterizacion_palabras['fgs_Nivel3'] == i].reset_index()
    df_texto['fgs_ObservacionCreacion'] = df_texto['fgs_ObservacionCreacion'].str.replace('\d+', '').replace('\br',' ')
    texto = text(df_texto['fgs_ObservacionCreacion'], dic)
    texto = texto.replace('\d+', '')
    print(i.upper(),' Total Registros: ',df_texto.shape[0])
    StopWords = SW_grupos + palabras_adicionales #palabras['StopWords'].str.split(' ')[0]
    wordcloud(texto, 'seis', StopWords)
    nivel3.append(i)
    palabras_clave.append(list(ngrams(texto, 1, 10, StopWords).to_frame().index))
    bigramas.append(list(ngrams(texto, 2, 10, StopWords).to_frame().index))
    trigramas.append(list(ngrams(texto, 3, 10, StopWords ).to_frame().index))
    print('-----------------------------------------------------------------------------------------------')
    

## **TABLA CON VALORES RESUMIDOS -CREACION**

In [ ]:
obs_creacion_ben = pd.DataFrame({'nivel3':nivel3,'palabras_clave':palabras_clave, 'bigramas':bigramas, 'trigramas':trigramas})
obs_creacion_ben

## **NUBES DE PALABRAS Y PALABRAS CLAVES POR NIVEL 3 - OBSERVACIONES DE SOLUCION**

### **NIVELES CON MAYOR VOLUMEN**

In [ ]:
df_total = beneficios
nivel3 = []
palabras_clave = []
bigramas = []
trigramas = []

for i in list(mayores_volumenes['nivel3'].unique()):
    df_texto=  df_total[df_total['fgs_Nivel3'] == i]
    palabras = caracterizacion_palabras[caracterizacion_palabras['fgs_Nivel3'] == i].reset_index()
    df_texto['fgs_ObservacionSolucion'] = df_texto['fgs_ObservacionSolucion'].str.replace('\d+', '')
    texto = text(df_texto['fgs_ObservacionSolucion'], dic)
    texto = texto.replace('br','').replace('\d+', '')
    print(i.upper(),' Total Registros: ',df_texto.shape[0])
    StopWords = palabras['StopWords'].str.split(' ')[0]
    wordcloud(texto, 'seis', StopWords)
    nivel3.append(i)
    palabras_clave.append(list(ngrams(texto, 1, 10, StopWords).to_frame().index))
    bigramas.append(list(ngrams(texto, 2, 10, StopWords).to_frame().index))
    trigramas.append(list(ngrams(texto, 3, 10, StopWords ).to_frame().index))
    print('-----------------------------------------------------------------------------------------------')



### **GRUPOS**

In [ ]:
for i in list(df_total['GRUPOS'].unique()):
    df_texto=  df_total[df_total['GRUPOS'] == i]
    df_texto['fgs_ObservacionSolucion'] = df_texto['fgs_ObservacionSolucion'].str.replace('\d+', '')
    #palabras = caracterizacion_palabras[caracterizacion_palabras['fgs_Nivel3'] == i].reset_index()
    texto = text(df_texto['fgs_ObservacionSolucion'], dic)
    texto = texto.replace('br','').replace('\d+', '')
    print(i.upper(),' Total Registros: ',df_texto.shape[0])
    StopWords = SW_grupos + palabras_adicionales #palabras['StopWords'].str.split(' ')[0]
    wordcloud(texto, 'seis', StopWords)
    nivel3.append(i)
    palabras_clave.append(list(ngrams(texto, 1, 10, StopWords).to_frame().index))
    bigramas.append(list(ngrams(texto, 2, 10, StopWords).to_frame().index))
    trigramas.append(list(ngrams(texto, 3, 10, StopWords ).to_frame().index))
    print('-----------------------------------------------------------------------------------------------')
    

## **TABLA CON VALORES RESUMIDOS -SOLUCION**

In [ ]:
obs_solucion_ben = pd.DataFrame({'nivel3':nivel3,'palabras_clave':palabras_clave, 'bigramas':bigramas, 'trigramas':trigramas})
obs_solucion_ben

In [ ]:
Base_general_extraida_P

## **NUBES DE PALABRAS Y PALABRAS CLAVES POR NIVEL 3 - OBSERVACIONES DE ESCALAMIENTO**

###**EXTRACCION**

In [ ]:
df_escalamiento = df[(~df['fgs_ObservacionEscalamientoN2'].isna())]
df_escalamiento['GRUPOS'] = df_escalamiento['fgs_Nivel3']
df_escalamiento['GRUPOS'].replace(dic_grupos, inplace=True)
df_escalamiento.shape

###**CANTIDAD DE OBSERVACIONES POR GRUPO Y POR NIVEL**

In [ ]:
df_escalamiento.shape

In [ ]:
df_escalamiento.fgs_Nivel3.value_counts()

In [ ]:
df_escalamiento.GRUPOS.value_counts()

In [ ]:
for i in list(df_escalamiento['GRUPOS'].unique()):
    df_texto=  df_escalamiento[df_escalamiento['GRUPOS'] == i]
    #palabras = caracterizacion_palabras[caracterizacion_palabras['fgs_Nivel3'] == i].reset_index()
    df_texto['fgs_ObservacionEscalamientoN2'] = df_texto['fgs_ObservacionEscalamientoN2'].str.replace('\d+', '')
    texto = text(df_texto['fgs_ObservacionEscalamientoN2'], dic)
    texto = texto.replace('br','').replace('\d+', '')
    print(i.upper(),' Total Registros: ',df_texto.shape[0])
    StopWords = SW_grupos + palabras_adicionales  #palabras['StopWords'].str.split(' ')[0]
    wordcloud(texto, 'seis', StopWords)
    nivel3.append(i)
    palabras_clave.append(list(ngrams(texto, 1, 10, StopWords).to_frame().index))
    bigramas.append(list(ngrams(texto, 2, 10, StopWords).to_frame().index))
    trigramas.append(list(ngrams(texto, 3, 10, StopWords ).to_frame().index))
    print('-----------------------------------------------------------------------------------------------')
    

# **EXPLORACION DE TEXTO - CUENTAS POR PAGAR**

## **NUBES DE PALABRAS Y PALABRAS CLAVES POR NIVEL 3 - OBSERVACIONES DE CREACION**

In [ ]:
Cuentas = pd.read_csv(ecopetrol_2)# cambiar por tabala actual.
#

In [ ]:
df_total = Cuentas
nivel3 = []
palabras_clave = []
bigramas = []
trigramas = []

for i in list(df_total.fgs_Nivel3.unique()):
    df_texto=  df_total[df_total['fgs_Nivel3'] == i]

    texto = text(df_texto['fgs_ObservacionCreacion'], dic)
    texto = texto.replace('br','')
    print(i.upper())
    StopWords = str(i).split(' ')
    wordcloud(texto, 'seis', StopWords)
    nivel3.append(i)
    palabras_clave.append(list(ngrams(texto, 1, 10).to_frame().index))
    bigramas.append(list(ngrams(texto, 2, 10).to_frame().index))
    trigramas.append(list(ngrams(texto, 3, 10).to_frame().index))
    print('-----------------------------------------------------------------------------------------------')
    

## **TABLA CON VALORES RESUMIDOS**

In [ ]:
obs_creacion_cuentas = pd.DataFrame({'nivel3':nivel3,'palabras_clave':palabras_clave, 'bigramas':bigramas, 'trigramas':trigramas})
obs_creacion_cuentas

## **NUBES DE PALABRAS Y PALABRAS CLAVES POR NIVEL 3 - OBSERVACIONES DE SOLUCION**

In [ ]:
df_total = Cuentas
nivel3 = []
palabras_clave = []
bigramas = []
trigramas = []

for i in list(df_total.fgs_Nivel3.unique()):
    df_texto=  df_total[df_total['fgs_Nivel3'] == i]

    texto = text(df_texto['fgs_ObservacionSolucion'], dic)
    print(i.upper())
    StopWords = str(i).split(' ')
    wordcloud(texto, 'seis', StopWords)
    nivel3.append(i)
    palabras_clave.append(list(ngrams(texto, 1, 30).to_frame().index))
    bigramas.append(list(ngrams(texto, 2, 20).to_frame().index))
    trigramas.append(list(ngrams(texto, 3, 20).to_frame().index))
    print('-----------------------------------------------------------------------------------------------')
    

## **TABLA CON VALORES RESUMIDOS -SOLUCION**

In [ ]:
obs_solucion_cuentas = pd.DataFrame({'nivel3':nivel3,'palabras_clave':palabras_clave, 'bigramas':bigramas, 'trigramas':trigramas})
obs_solucion_cuentas

In [ ]:
with pd.ExcelWriter('Caracterizacion_inicial.xlsx') as writer:  
    obs_creacion_cuentas.to_excel(writer, sheet_name='Cuentas-obsCreacion', index =  False)
    obs_solucion_cuentas.to_excel(writer, sheet_name='Cuentas-obsSolucion', index = False)
    obs_creacion_ben.to_excel(writer, sheet_name='Beneficios-obsCreacion', index = False)
    obs_solucion_ben.to_excel(writer, sheet_name='Beneficios-obsSolucion', index = False)

# **DISTRIBUCIÓN DE PALABRAS Y REGISTROS**

## **Cuentas por pagar**

In [ ]:
Cuentas['# Palabras_ObservacionCreacion'] = Cuentas['fgs_ObservacionCreacion'].str.split().str.len()
Cuentas['# Palabras_ObservacionSolucion'] = Cuentas['fgs_ObservacionSolucion'].str.split().str.len()

**Cantidad de registros**

In [ ]:
Cuentas.fgs_Nivel3.value_counts()

In [ ]:
Palabras_cuentas = Cuentas[['fgs_Nivel3','# Palabras_ObservacionCreacion', '# Palabras_ObservacionSolucion']]

#descripcion_observ = df_OF_relevante[['# Palabras_fgs_ObservacionCreacion', '# Palabras_fgs_ObservacionSolucion','# Palabras_fgs_ObservacionCierre' ]].describe()
for i in list(Cuentas.fgs_Nivel3.unique()):
    Palabras_cuentas = Cuentas[Cuentas['fgs_Nivel3']==i][['# Palabras_ObservacionCreacion', '# Palabras_ObservacionSolucion']]
    describe_var = Palabras_cuentas.describe().round(1)
    describe_var = describe_var.append(pd.Series(Palabras_cuentas.var(), name = 'Variance')) 
    describe_var = describe_var.append(pd.Series(Palabras_cuentas.sum(), name = 'Sum'))
    print('--------------------------------------------------------------------------------------')
    print(i.upper())
    print(describe_var.round(1))

## **Beneficios educativos**

In [ ]:
beneficios['# Palabras_ObservacionCreacion'] = beneficios['fgs_ObservacionCreacion'].str.split().str.len()
beneficios['# Palabras_ObservacionSolucion'] = beneficios['fgs_ObservacionSolucion'].str.split().str.len()

**Cantidad de registros**

In [ ]:
beneficios.fgs_Nivel3.value_counts()

In [ ]:
Palabras_beneficios = beneficios[['fgs_Nivel2','fgs_Nivel3','# Palabras_ObservacionCreacion', '# Palabras_ObservacionSolucion']]


In [ ]:
#descripcion_observ = df_OF_relevante[['# Palabras_fgs_ObservacionCreacion', '# Palabras_fgs_ObservacionSolucion','# Palabras_fgs_ObservacionCierre' ]].describe()
for i in list(beneficios.fgs_Nivel3.unique()):
    Palabras_beneficios = beneficios[(beneficios['fgs_Nivel3']==i)][['# Palabras_ObservacionCreacion', '# Palabras_ObservacionSolucion']]
    describe_var = Palabras_beneficios.describe().round(1)
    describe_var = describe_var.append(pd.Series(Palabras_beneficios.var(), name = 'Variance')) 
    describe_var = describe_var.append(pd.Series(Palabras_beneficios.sum(), name = 'Sum')) 
    print('--------------------------------------------------------------------------------------')
    print(i.upper())
    print(describe_var.round(1))